## 1. Typed Dictionary

In [1]:
from typing import TypedDict

class Movie(TypedDict):
    name : str
    year : int

movie = Movie(name="Avengers", year=2019)

- **Type Safety:** Type Definition for each key value
- **Enhanced Readability:** Makes debugging easier and code readable

## 2. Union
- Any of type specified: ``Union[<type1>, <type2>]``
- Any type except specified is invalid.

In [2]:
from typing import Union

def square(x: Union[int, float]) -> float:
    return x * x

## 3. Optional
- Optional argument
- But also can have type limit if specified.

In [3]:
from typing import Optional

def greet(name: Optional[str]) -> None:
    if name is None:
        print("Hey random persion.")
    else:
        print(f"Hey {name}")

## 4. Any
- Of type any.
- No type restriction

In [4]:
from typing import Any

def print_value(x: Any):
    print(x)

## 5. Annotated
---
**Purpose:**
- Attach metadata to a type, without changing the type.
- No runtime effect on variable, but internals of some framework use it for different purpose.
- Used by frameworks like:
    - **Pydantic:** for validators and UI docs in swagger
    - **FastAPI:** to extract info for request validation
    - **LangGraph:** for reducers like add_messages, last_value

---

**General usage:**
```python
from typing import Annotated

x: Annotated[int, "positive only"]

```
- This means: "x is an int — and also has metadata 'positive only'".

- Tools (like LangGraph, Pydantic, etc.) can read that metadata to do extra things (e.g., validation, logging, injection).
---

**Usage with FastAPI:**

```python
from typing import Annotated
from fastapi import Query

def search(q: Annotated[str, Query(min_length=3)]):
    ...
```

Here:
- q is a str
-But FastAPI uses the Query() metadata to validate length
---

**Note that annotated metadata can also be a function, how annotation is used depends on how internals of framework work.**

In [5]:
from typing import Annotated

x =  Annotated[int, "some extra metadata"]

x.__metadata__

('some extra metadata',)

## 6. Sequence:

``Sequence[T]`` is an abstract base class that:
- represents a **read-only**, 
- **ordered collection** 
- of elements of type T.

**Examples of types that are Sequence:**
- list
- tuple
- str (technically a sequence of characters)

**List Vs Sequence:**

| `list[T]`        | `Sequence[T]`                        |
| ---------------- | ------------------------------------ |
| Mutable          | Immutable (interface only)           |
| Concrete         | Abstract                             |
| Can `.append()`  | Cannot `.append()` (interface level) |
| e.g. `List[int]` | e.g. `Sequence[int]`                 |


In [6]:
from typing import Sequence

def print_all(items: Sequence[str]):
    for item in items:
        print(item)

print_all(["a", "b", "c"])      # ✅ list
print("\n\n")
print_all(("x", "y", "z"))      # ✅ tuple
print("\n\n")
print_all("hello")             # ✅ str (sequence of characters)

a
b
c



x
y
z



h
e
l
l
o


## Use Case: Annotated with Reducers in Langgraph: 

**``langchain_core.messages.add_mesages``**

---
```python
from typing import Annotated, Sequence
from langgraph.graph import State, add_messages
from langchain_core.messages import BaseMessage

class AgentState(State):
    messages: Annotated[Sequence[BaseMessage], add_messages]
```
Tells LangGraph to append new messages rather than overwrite

---

**``BaseMessage``**

- Note that, every message is a instance (implementation) of  ``BaseMessage``

```txt
BaseMessage  # abstract base class
├── HumanMessage
├── AIMessage
├── SystemMessage
├── FunctionMessage
├── ToolMessage
├── ChatMessage  # (used for custom roles)
```
---

**``messages:``**

```python
messages: Annotated[Sequence[BaseMessage], add_messages]
```



In [8]:
from typing import Annotated, Sequence, TypedDict
from langgraph.graph import StateGraph, add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

# Step 1: Define State
class ChatState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# Step 2: Simple Node
def echo_node(state: ChatState) -> ChatState:
    last_user_msg = next((m.content for m in reversed(state["messages"]) if m.type == "human"), "...")
    ai_response = AIMessage(content=f"Echo: {last_user_msg}")
    return {"messages": [ai_response]}

# Step 3: Define Graph
graph = StateGraph(ChatState)
graph.add_node("echo", echo_node)
graph.set_entry_point("echo")
echo_graph = graph.compile()

# Step 4: Initial State
initial_state = {
    "messages": [HumanMessage(content="Hello")]
}

# Step 5: Run the Graph (1st turn)
state1 = echo_graph.invoke(initial_state)
print("--- After 1st Run ---")
for m in state1["messages"]:
    print(f"{m.type}: {m.content}")

# Step 6: Run Again with More Input (2nd turn)
state2 = echo_graph.invoke({
    "messages": state1["messages"] + [HumanMessage(content="How are you?")]
})
print("\n--- After 2nd Run ---")
for m in state2["messages"]:
    print(f"{m.type}: {m.content}")


--- After 1st Run ---
human: Hello
ai: Echo: Hello

--- After 2nd Run ---
human: Hello
ai: Echo: Hello
human: How are you?
ai: Echo: How are you?


```python
# Step 2: Simple Node
def echo_node(state: ChatState) -> ChatState:
    last_user_msg = next((m.content for m in reversed(state["messages"]) if m.type == "human"), "...")
    ai_response = AIMessage(content=f"Echo: {last_user_msg}")
    return {"messages": [ai_response]}
```

In the above code:
- Instead of returning whole state, we are returning state to be updated / appended only.
- It doesnt rewrites whole state, instead reduces.
---

**Other naive approach without reducer would be:**

```python
class ChatState(TypedDict):
    messages: ...

def echo_node(state: ChatState) -> ChatState:
    state["<key>"] = "<value>"
    return state
```

- Here, we are returning a whole state, by updating a single key in a state.
---